In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import  LabelEncoder

<h3>Загрузка датасета</h3>

In [86]:
#загрузка датасета
df = pd.read_csv('coords_data_numerated.csv')
df = df.drop(columns=['Unnamed: 0', 'GSR'])
df.head()

,id,BreathingType,TimeStamp_sec,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
0,1,0,0.058895,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0,0.111889,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0,0.158371,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0,0.262516,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0,0.295620,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


In [87]:
y = df['BreathingType']
X = df.drop(columns=['BreathingType','TimeStamp_sec'])
X.tail()

,id,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
230921,258,1.063,-1.649,3.843,1.093,-1.839,3.624,1.030,-1.867,3.859
230922,258,1.062,-1.648,3.842,1.094,-1.841,3.625,1.030,-1.867,3.857
230923,258,1.062,-1.648,3.842,1.093,-1.841,3.624,1.030,-1.868,3.855
230924,258,1.063,-1.647,3.842,1.094,-1.841,3.624,1.029,-1.868,3.853
230925,258,1.063,-1.647,3.842,1.094,-1.841,3.623,1.030,-1.868,3.853


In [88]:
#переименование столбцов
dict_renames = {
    'FirstMarkerXCoord' : 'FMX',
    'FirstMarkerYCoord' : 'FMY',
    'FirstMarkerZCoord' : 'FMZ',
    'SecondMarkerXCoord' :  'SMX',
    'SecondMarkerYCoord' : 'SMY',
    'SecondMarkerZCoord' : 'SMZ',
    'ThirdMarkerXCoord' : 'TMX',
    'ThirdMarkerYCoord' : 'TMY',
    'ThirdMarkerZCoord' : 'TMZ'
}
X = X.rename(columns=dict_renames)
X.head()

,id,FMX,FMY,FMZ,SMX,SMY,SMZ,TMX,TMY,TMZ
0,1,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


<h3>Предобработка датасета</h3>

In [89]:
#удаление лишних временных рядов
X = X[X['id'] != 27]
X = X[X['id'] != 24]
X = X.reset_index(drop=True)
X.head()

,id,FMX,FMY,FMZ,SMX,SMY,SMZ,TMX,TMY,TMZ
0,1,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


In [90]:
#приведение к одинаковому количеству точек для временных рядов
numeric_cols = X.drop(columns=['id'])
X2 = X.iloc[0:0]
for i in range(1, 259):
    if i in [24, 27]:
        continue
    X_id = X[X['id'] == i].reset_index(drop=True)
    for col in numeric_cols:
        X_id = X_id[:900]
        X_id = X_id.reset_index(drop=True)
    X2 = pd.concat([X2, X_id], axis=0, ignore_index=True)

In [91]:
X = X2

<h3>Вычисление новых признаков</h3>

In [92]:
#расчет новых преобразований
X2 = X.iloc[0:0]
for i in range(1, 259):
    if i in [24, 27]:
        continue
    X_id = X[X['id'] == i].reset_index(drop=True)
    for col in numeric_cols:
        X_id[col+'_roll10_mean'] = X_id[col].rolling(10).mean()
        X_id[col+'_roll10_median'] = X_id[col].rolling(10).median()
        X_id[col+'_change10'] = X_id[col].diff(periods=10)
        X_id[col+'_pct10'] = X_id[col].pct_change(periods=10)


        X_id = X_id[10:]
        X_id = X_id.reset_index(drop=True)
    X2 =  pd.concat([X2, X_id], axis=0, ignore_index=True)

In [93]:
X = X2

<h3>Подготовка данных для библиотеки TSLearn</h3>

In [95]:
#преобразование датасета для библиотеки sktime
numeric_cols = X.drop(columns='id').columns
X_3d = []
for id in range(1, 259):
    if id in [24, 27]:
        continue
    X_id = X[X['id'] == id]
    instance_features = []
    for col in numeric_cols:
        lst = X_id[col].to_list()
        instance_features.append(lst)
    X_3d.append(instance_features)
X_3d = np.array(X_3d)

In [96]:
#преобразование датасета для библиотеки sktime
y_3d = []
for id in range(1, 259):
    if id in [24, 27]:
        continue
    df_id = df[df['id'] == id]
    y_3d.append(df_id['BreathingType'].iloc[0])
y_3d = np.array(y_3d)

In [97]:
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
#расчет метрик по тесовой выборке
def calculate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-score: ", f1)
    report = classification_report(y_test, y_pred)
    print(report)

In [98]:
#разделение на обучающую(80%) и тестовую (20%) выборки
X_train, X_test, y_train, y_test = train_test_split(X_3d, y_3d, test_size=0.3, random_state=42)

<h3>Тестирование моделей классификации временных рядов</h3>

<h4>KNeighbors Time Series Classifier </h4>

In [99]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

In [100]:
knn = KNeighborsTimeSeriesClassifier(n_neighbors=2)

In [101]:
details = knn.fit(X_train, y_train)

In [102]:
y_pred = knn.predict(X_test)
calculate_metrics(y_test, y_pred)

Accuracy:  0.19480519480519481
Recall:  0.19480519480519481
Precision:  0.12763706881353942
F1-score:  0.15284271284271286
              precision    recall  f1-score   support

           0       0.24      0.40      0.30        30
           1       0.12      0.12      0.12        24
           2       0.00      0.00      0.00        23

    accuracy                           0.19        77
   macro avg       0.12      0.18      0.14        77
weighted avg       0.13      0.19      0.15        77



C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

<h3>Time Series SVC</h3>

In [103]:
from tslearn.svm import TimeSeriesSVC

clf = TimeSeriesSVC(C=1.0, kernel="gak")

In [104]:
details = clf.fit(X_train, y_train)

In [105]:
y_pred = clf.predict(X_test)
calculate_metrics(y_test, y_pred)

Accuracy:  0.3116883116883117
Recall:  0.3116883116883117
Precision:  0.09714960364311014
F1-score:  0.1481290986241481
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        30
           1       0.31      1.00      0.48        24
           2       0.00      0.00      0.00        23

    accuracy                           0.31        77
   macro avg       0.10      0.33      0.16        77
weighted avg       0.10      0.31      0.15        77



C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

<h3>Early Time Series Classification</h3>

In [106]:
from tslearn.early_classification import NonMyopicEarlyClassifier

model = NonMyopicEarlyClassifier(n_clusters=3, lamb=1000, cost_time_parameter=.1, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [108]:
preds, pred_times = model.predict_class_and_earliness(X_test)

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\tslearn\early_classification\early_classification.py:282: RuntimeWarning: overflow encountered in exp
  s_k = 1. / (1. + np.exp(-self.lamb * delta_k))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\tslearn\early_classification\early_classification.py:282: RuntimeWarning: overflow encountered in exp
  s_k = 1. / (1. + np.exp(-self.lamb * delta_k))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\tslearn\early_classification\early_classification.py:282: RuntimeWarning: overflow encountered in exp
  s_k = 1. / (1. + np.exp(-self.lamb * delta_k))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\tslearn\early_classification\early_classification.py:282: RuntimeWarning: overflow encountered in exp
  s_k = 1. / (1. + np.exp(-self.lamb * delta_k))
C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\tslearn\early_classificati

In [109]:
calculate_metrics(y_test, preds)

Accuracy:  0.2077922077922078
Recall:  0.2077922077922078
Precision:  0.19041252864782277
F1-score:  0.19772584824243075
              precision    recall  f1-score   support

           0       0.25      0.30      0.27        30
           1       0.06      0.04      0.05        24
           2       0.25      0.26      0.26        23

    accuracy                           0.21        77
   macro avg       0.19      0.20      0.19        77
weighted avg       0.19      0.21      0.20        77



<h3>MultiLayer Perceptron Classifier</h3>

In [110]:
from tslearn.neural_network import TimeSeriesMLPClassifier

mlp = TimeSeriesMLPClassifier(hidden_layer_sizes=(64, 64), random_state=42, verbose=True, early_stopping=False, n_iter_no_change=100)

In [111]:
details = mlp.fit(X_train, y_train, )

Iteration 1, loss = 1.24672001
Iteration 2, loss = 23.48228037
Iteration 3, loss = 17.19694870
Iteration 4, loss = 11.71803646
Iteration 5, loss = 5.56302885
Iteration 6, loss = 7.37327664
Iteration 7, loss = 13.24263985
Iteration 8, loss = 5.28592851
Iteration 9, loss = 4.22347315
Iteration 10, loss = 6.65139356
Iteration 11, loss = 7.98064955
Iteration 12, loss = 5.17962726
Iteration 13, loss = 5.46473835
Iteration 14, loss = 2.92875747
Iteration 15, loss = 7.85832013
Iteration 16, loss = 8.12258142
Iteration 17, loss = 4.40353090
Iteration 18, loss = 4.13761785
Iteration 19, loss = 3.91435002
Iteration 20, loss = 6.87490071
Iteration 21, loss = 7.59856035
Iteration 22, loss = 4.80299044
Iteration 23, loss = 4.75744973
Iteration 24, loss = 4.41075137
Iteration 25, loss = 4.21496917
Iteration 26, loss = 5.14553904
Iteration 27, loss = 3.05294739
Iteration 28, loss = 2.15022642
Iteration 29, loss = 3.01232484
Iteration 30, loss = 3.31188007
Iteration 31, loss = 3.58396416
Iteration 32,

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [112]:
y_pred = mlp.predict(X_test)
calculate_metrics(y_test, y_pred)

Accuracy:  0.2727272727272727
Recall:  0.2727272727272727
Precision:  0.2603292924148539
F1-score:  0.2572941056703655
              precision    recall  f1-score   support

           0       0.24      0.13      0.17        30
           1       0.36      0.50      0.42        24
           2       0.19      0.22      0.20        23

    accuracy                           0.27        77
   macro avg       0.26      0.28      0.26        77
weighted avg       0.26      0.27      0.26        77

